In [117]:
# Pendientes
# Remove values during field campaigns (escala diaria)
# Repetir para los pozos del sitio sur

# Estrategia: Generar series temporales a escala diaria para uso de todos en el proyecto.

In [118]:
# Importar las bibliotecas necesarias
import pandas as pd

In [119]:
# Definir la ruta de los datos

# Sitio norte
fp_sdh1_ps01 = 'data/raw/piezometers/Data_15_07_2025_ SDH1PS01_COMPENSADA.xlsx'
fp_sdh1_ps02 = 'data/raw/piezometers/Data_15_07_2025_ SDH1PS02_COMPENSADA.xlsx'

In [120]:
# Leer los datos

# Sitio norte
sdh1_ps01 = pd.read_excel(fp_sdh1_ps01)
sdh1_ps02 = pd.read_excel(fp_sdh1_ps02)

In [121]:
# Función para crear una columna 'Timestamps' y usarla como indice con formato datetime

def timestamp_as_index(dataframe):

    # Convertir las columnas 'Date' y 'Time' en strings
    date_str = dataframe['Date'].astype(str)
    time_str = dataframe['Time'].astype(str)

    # Crear columna 'Timestamps' con formato datetime
    dataframe['Timestamps'] = pd.to_datetime(
        date_str + ' ' + time_str,
        format='%Y-%m-%d %H:%M:%S'
    )

    # Establecer 'Timestamps' como indice
    dataframe = dataframe.set_index('Timestamps')
    
    # Eliminar columnas 'Date' y 'Time'
    dataframe = dataframe.drop(columns=['Date', 'Time'])

    return dataframe

In [122]:
# Ejecutar funcion para establecer 'Timestamps' como indice
sdh1_ps01 = timestamp_as_index(sdh1_ps01)
sdh1_ps02 = timestamp_as_index(sdh1_ps02)

In [123]:
# Diccionario con nombres de columnas originales y renombradas
columns_dict = {
    'TEMPERATURE' : 'Temperature_C',
    'NE_m' : 'Depth_m',
    'Cota_m' : 'Static_level_masl'
}

# Lista con nombres de columnas no necesarias
columns_list = [
    'ms',
    'LEVEL',
    'P_baro'
]

In [124]:
# Funcion para renombrar columnas y eliminar las innecesarias

def format_columns(df):
    df = df.rename(columns=columns_dict).drop(columns=columns_list, axis=1, errors='ignore')
    return df

In [125]:
# Ejecutar la función de formateo de columnas

sdh1_ps01 = format_columns(sdh1_ps01)
sdh1_ps02 = format_columns(sdh1_ps02)

In [126]:
# Función para detectar datos duplicados
def check_duplicates(df, df_name):

    # Hace una copia del df
    df_copy = df.copy()

    # Genera una columna que almacena los registros duplicados como True
    df_copy['duplicated'] = df_copy.index.duplicated(keep=False)

    # Filtra la copia del df, manteniendo sólo los registros duplicados
    dup_filter = df_copy[df_copy['duplicated'] == True]

    # Si los hay, imprime cuántos y cuáles registros están duplicados
    if dup_filter.empty:
        print(f'No hay datos duplicados en {df_name}')
    else:
        print(f'{df_name} tiene {len(dup_filter)} datos duplicados:')
        print(dup_filter)

In [127]:
# Aplicar función de detección de duplicados
check_duplicates(sdh1_ps01, 'sdh_ps01')
check_duplicates(sdh1_ps02, 'sdh_ps02')

No hay datos duplicados en sdh_ps01
No hay datos duplicados en sdh_ps02


In [128]:
# Función para detectar intervalos de medición y/o saltos en los datos
def check_discontinuities(df, df_name, max_expected_interval):

    # Hace una copia del df
    df_copy = df.copy()

    # Genera una columna que almacena los intervalos entre datos como Timedelta
    df_copy['diff'] = df_copy.index.diff()

    # Genera un objeto con el conteo de valores en la columna diff
    value_count = df_copy['diff'].value_counts()

    # Genera un objeto con los registros con intervalos superiores a los esperados
    discontinuities = df_copy[df_copy['diff'] > pd.Timedelta(max_expected_interval)]

    # Imprime el resultado del conteo de valores
    print(f'Los intervalos entre datos en {df_name} son: {value_count}')
    print('')

    # Imprime los registros que superan el intervalo máximo esperado
    print(f'Los intervalos anómalos en {df_name} son: {discontinuities}')
    print('')

In [129]:
check_discontinuities(sdh1_ps01, 'sdh1_ps01', '15min')
check_discontinuities(sdh1_ps02, 'sdh1_ps02', '15min')

Los intervalos entre datos en sdh1_ps01 son: diff
0 days 00:10:00    35535
0 days 00:05:00    18344
0 days 00:15:00    10346
0 days 00:40:00        2
0 days 01:40:00        1
0 days 01:30:00        1
Name: count, dtype: int64

Los intervalos anómalos en sdh1_ps01 son:                      Temperature_C  Depth_m  Static_level_masl            diff
Timestamps                                                                    
2024-07-25 18:20:00          8.767   0.5355          3827.0645 0 days 01:40:00
2024-09-05 14:00:00          5.560   0.1712          3827.4288 0 days 00:40:00
2024-11-10 15:00:00          6.464   0.5266          3827.0734 0 days 01:30:00
2025-01-22 14:00:00          9.608   0.5726          3827.0274 0 days 00:40:00

Los intervalos entre datos en sdh1_ps02 son: diff
0 days 00:10:00    35530
0 days 00:20:00        1
Name: count, dtype: int64

Los intervalos anómalos en sdh1_ps02 son:                      Temperature_C  Depth_m  Static_level_masl            diff
Timestam

In [190]:
# Fechas de terreno. Sólo se incluyen fechas de terreno efectivo (en el SDH)
# La fecha de término es excluyente
may_2024 = pd.date_range(start='2024-05-21', end='2024-05-23')
jul_2024 = pd.date_range(start='2024-07-25', end='2024-07-28')
sep_2024 = pd.date_range(start='2024-09-03', end='2024-09-07')
nov_2024 = pd.date_range(start='2024-11-05', end='2024-11-12')
jan_2025 = pd.date_range(start='2025-01-21', end='2025-01-23')
apr_2025 = pd.date_range(start='2025-04-28', end='2025-05-02')
jul_2025 = pd.date_range(start='2025-07-08', end='2025-07-16')


In [199]:
sdh1_ps01.loc[sep_2024.min() : sep_2024.max()].sort_values(by='Temperature_C', ascending=False).head(10)

,Temperature_C,Depth_m,Static_level_masl
Timestamps,,,
2024-09-05 14:00:00,5.560,0.1712,3827.4288
2024-09-05 14:15:00,5.444,0.1735,3827.4265
2024-09-05 14:30:00,5.389,0.1741,3827.4259
2024-09-05 14:45:00,5.358,0.1737,3827.4263
2024-09-05 15:00:00,5.338,0.1749,3827.4251
2024-09-05 15:15:00,5.328,0.1755,3827.4245
2024-09-05 15:30:00,5.320,0.1757,3827.4243
2024-09-05 15:45:00,5.312,0.1759,3827.4241
2024-09-05 16:00:00,5.308,0.1770,3827.4230


In [133]:
# sdh1_ps01['diff'] = sdh1_ps01.index.diff()

# intervals_to_process = ['5min', '10min', '15min']
# reindexed_parts = []

# for interval in intervals_to_process:

#     current_interval = sdh1_ps01[sdh1_ps01['diff'] == pd.Timedelta(interval)]

#     if not current_interval.empty:
#         full_index = pd.date_range(
#             start=current_interval.index.min(),
#             end=current_interval.index.max(),
#             freq=interval 
#         )
        
#         reindexed = current_interval.reindex(full_index)
#         reindexed_parts.append(reindexed)

# df_unificado = pd.concat(reindexed_parts).sort_index()